In [1]:
import pandas as pd
import pickle
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import contractions
import numpy as np

In [2]:
tfidf = pickle.load(open('vectorizer1.pkl','rb'))
clf = pickle.load(open('modelc.pkl','rb'))

In [3]:
def return_prob(text):
    soup = BeautifulSoup(text, 'html.parser')
    text=soup.get_text()
    text = contractions.fix(text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    tokens = word_tokenize(text.lower())

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the remaining words
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Join the lemmatized tokens back into a single string
    text = ' '.join(lemmatized_tokens)
    vec=[]
    vec.append(text)
    vector=tfidf.transform(vec)
    sentiment_prob = clf.predict_proba(vector)[:, 1][0]
    vec.clear()
    return sentiment_prob

In [4]:
return_prob("Wonderful movie of all time")

0.9986640696080435

In [5]:
from tmdbv3api import TMDb
from tmdbv3api import Movie

# Replace 'YOUR_API_KEY' with your actual TMDb API key
tmdb = TMDb()
tmdb.api_key = 'f03abce17e11e695cce8ce75b3d4348d'

In [6]:
movie = Movie()

In [16]:
def getSentiment(title):
    allreviews=[]
    title = title.split('(')[0].strip()
    results = movie.search(title)
    if len(results) > 0:
        # Get the first search result
        movie_id = results[0].id
        reviews = movie.reviews(movie_id)
        for review in reviews:
            allreviews.append(review.content)
    else:
        return 0
    sum=0
    try:
        if len(allreviews)>10:
            allreviews=allreviews[:10]
        for i in allreviews:
            sum = sum+return_prob(i)
        score=sum/len(allreviews)
        return score
    except:
        return 0
    
    

In [23]:
getSentiment('Crocodile Dundee (1986)')

0.6323236492061749